In [1]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#!python3 -m pip install pyreadstat

In [3]:
text_to_val_df = pd.read_excel("Variablenwerte.xls", names=["Name", "Value", "Text"], header=1)
# Fill all names
for i in range(len(text_to_val_df)):
    if pd.isna(text_to_val_df.loc[i, "Name"]):
        text_to_val_df.loc[i, "Name"] = text_to_val_df.loc[i - 1, "Name"]
# make values ints
for i in range(len(text_to_val_df)):
    val = text_to_val_df.loc[i, "Value"]
    if val == ",00":
        val = 0
    elif val == "1,00":
        val = 1
    else:
        val = str(val).split(",")[0].replace(",", "")
        if val == "":
            print(i)
            val = -1
        else:
            val = int(val)
    text_to_val_df.loc[i, "Value"] = val
text_to_val_df

,Name,Value,Text
0,f1a_1,0,"no Ja, jemand mit positivem Test"
1,f1a_1,1,"Ja, jemand mit positivem Test"
2,f1a_2,0,"no Ja, jemand mit ärztlicher Diagnose, aber o..."
3,f1a_2,1,"Ja, jemand mit ärztlicher Diagnose, aber ohne ..."
4,f1a_3,0,"no Ja, jemand mit möglichen Symptomen, aber n..."
...,...,...,...
1209,bl,12,Brandenburg
1210,bl,13,Mecklenburg-Vorpommern
1211,bl,14,Sachsen
1212,bl,15,Sachsen-Anhalt


In [4]:
df = pd.read_spss("f20.0251z_290620.sav")
# replace strings by values using Variablenwerte.xls
replace_dict = {name: {row["Text"]: row["Value"] for _, row in text_to_val_df[text_to_val_df["Name"] == name].iterrows()} for name in text_to_val_df["Name"].unique()}
df = df.replace(replace_dict)
# replace empty rows by NaN
df = df.replace({"": np.nan, " ": np.nan})
# remove "offen" fields
df = df[[col for col in df if "offen" not in col]]
df

,sernr,f1a_1,f1a_2,f1a_3,f1a_4,f1a_5,f1a_6,f1b_1,f1b_2,f1b_3,...,f145,f146,altq,altqx,ges,plz,bl,regbez,lkrs,weight
0,1.0,0,0,0,0,1,0,0,0,0,...,NaN,2,67.0,5,2,972.0,9,6.0,77.0,0.947
1,2.0,0,0,0,0,1,0,0,0,0,...,NaN,2,54.0,4,2,844.0,9,1.0,77.0,0.880
2,3.0,0,0,0,0,1,0,0,0,0,...,NaN,2,67.0,5,2,985.0,16,0.0,66.0,1.051
3,4.0,0,0,0,0,1,0,0,0,0,...,NaN,2,26.0,2,1,453.0,5,1.0,13.0,2.233
4,6.0,0,0,0,0,1,0,0,0,0,...,NaN,2,50.0,4,1,703.0,8,1.0,11.0,0.923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30063,31676.0,0,0,0,0,1,0,0,0,0,...,NaN,1,53.0,4,1,168.0,12,0.0,68.0,0.853
30064,31677.0,0,0,1,0,0,0,0,0,1,...,NaN,2,31.0,2,1,732.0,8,1.0,16.0,1.015
30065,31678.0,0,0,0,0,0,1,0,0,0,...,NaN,2,44.0,3,2,42.0,14,7.0,13.0,0.787
30066,31679.0,0,0,0,0,1,0,0,0,0,...,NaN,2,41.0,3,1,11.0,14,6.0,12.0,0.870


In [5]:
# detect all non-int and non-float cols
for col in df:
    if df[col].dtype not in ("float64", "int64"):
        print(col, df[col].dtype)
        non_na = df[col][~df[col].isna()]
        print("Len: ", len(non_na))
        print(non_na.head(3))
        if len(non_na.unique()) < 10:
            print("unique distrs:")
            for val in non_na.unique():
                print(val)
                print((non_na == val).mean())
        print()

nf33a category
Len:  30068
0    1.0
1    3.0
2    2.0
Name: nf33a, dtype: category
Categories (27, float64): [0.0, 1.0, 2.0, 3.0, ..., 32.0, 34.0, 36.0, 99.0]

nf33b category
Len:  30068
0    2.0
1    0.0
2    4.0
Name: nf33b, dtype: category
Categories (10, float64): [0.0, 1.0, 2.0, 3.0, ..., 6.0, 7.0, 8.0, 99.0]

nf34a category
Len:  30068
0     2.0
1     1.0
2    99.0
Name: nf34a, dtype: category
Categories (27, float64): [0.0, 1.0, 2.0, 3.0, ..., 30.0, 32.0, 36.0, 99.0]

nf34b category
Len:  30068
0     0.0
1     0.0
2    99.0
Name: nf34b, dtype: category
Categories (10, float64): [0.0, 1.0, 2.0, 3.0, ..., 6.0, 7.0, 8.0, 99.0]

nf35a category
Len:  30068
0     4.0
1    12.0
2     5.0
Name: nf35a, dtype: category
Categories (31, float64): [0.0, 1.0, 2.0, 3.0, ..., 31.0, 32.0, 36.0, 99.0]

nf35b category
Len:  30068
0    0.0
1    0.0
2    0.0
Name: nf35b, dtype: category
Categories (10, float64): [0.0, 1.0, 2.0, 3.0, ..., 6.0, 7.0, 8.0, 99.0]

nf36a category
Len:  30068
0     4.0
1  

/home/anton/.local/lib/python3.8/site-packages/pandas/io/formats/format.py:1403: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/home/anton/.local/lib/python3.8/site-packages/pandas/io/formats/format.py:1403: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/home/anton/.local/lib/python3.8/site-packages/pandas/io/formats/format.py:1403: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/home/anton/.local/lib/python3.8/site-packages/pandas/io/formats/format.py:1403: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/home/anton/.local/lib/p

In [6]:
# convert rows to ints
int_names = ["nf33a", "nf33b", "nf34a", "nf34b", "nf35a", "nf35b", "nf36a", "nf36b"]
for name in int_names:
    df[name] = df[name].astype(float)
# some have missings that cannot be converted to int
float_names = ["f41", "nf42", "f49", "nf76", "nf77", "nf82", "nf83", "nf84", "nf88", "nf92", "nf94", "nt1", "nt2", "f145", "altq"]
for name in float_names:
    df[name] = df[name].astype(float)
    #non_na_mask = ~df[name].isna()
    #df.loc[non_na_mask, name] = df.loc[non_na_mask, name].astype(float)

In [7]:
df

,sernr,f1a_1,f1a_2,f1a_3,f1a_4,f1a_5,f1a_6,f1b_1,f1b_2,f1b_3,...,f145,f146,altq,altqx,ges,plz,bl,regbez,lkrs,weight
0,1.0,0,0,0,0,1,0,0,0,0,...,NaN,2,67.0,5,2,972.0,9,6.0,77.0,0.947
1,2.0,0,0,0,0,1,0,0,0,0,...,NaN,2,54.0,4,2,844.0,9,1.0,77.0,0.880
2,3.0,0,0,0,0,1,0,0,0,0,...,NaN,2,67.0,5,2,985.0,16,0.0,66.0,1.051
3,4.0,0,0,0,0,1,0,0,0,0,...,NaN,2,26.0,2,1,453.0,5,1.0,13.0,2.233
4,6.0,0,0,0,0,1,0,0,0,0,...,NaN,2,50.0,4,1,703.0,8,1.0,11.0,0.923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30063,31676.0,0,0,0,0,1,0,0,0,0,...,NaN,1,53.0,4,1,168.0,12,0.0,68.0,0.853
30064,31677.0,0,0,1,0,0,0,0,0,1,...,NaN,2,31.0,2,1,732.0,8,1.0,16.0,1.015
30065,31678.0,0,0,0,0,0,1,0,0,0,...,NaN,2,44.0,3,2,42.0,14,7.0,13.0,0.787
30066,31679.0,0,0,0,0,1,0,0,0,0,...,NaN,2,41.0,3,1,11.0,14,6.0,12.0,0.870


In [8]:
df["f1a_1"].sum() 

559

In [9]:
df["f1a_2"].sum()

77

In [10]:
df["f1a_3"].sum()

494

In [11]:
df["f1a_4"].sum()

248

In [12]:
(df["f1a_1"] + df["f1a_2"] + df["f1a_3"] + df["f1a_4"]).astype(bool).sum()

1286

In [14]:
(df["f1b_1"] + df["f1b_2"] + df["f1b_3"] + df["f1b_4"]).astype(bool).sum()

3520

In [15]:
(df["f1a_1"] + df["f1b_1"]).astype(bool).sum()

1961

In [28]:
(df["f2a"] == 2).sum()

17

In [29]:
(df["f2b"] == 2).sum()

82

In [34]:
((df["f2a"] == 1).astype(int) + (df["f2b"] == 1).astype(int) + df["f1a_1"] + df["f1b_1"]).astype(bool).sum()

1988

In [187]:
len(df) - (28080 + 1906)

82

In [186]:
df

29986

In [188]:
len(df)

30068

In [35]:
pred_df = df.copy()

In [37]:
pred_df["target"] = ((df["f2a"] == 1).astype(int) + (df["f2b"] == 1).astype(int) + df["f1a_1"] + df["f1b_1"]).astype(bool)
drop_cols = [col for col in pred_df if "f1a" in col or "f2a" in col or "f2b" in col or "f1b" in col]
pred_df = pred_df.drop(columns=drop_cols)

In [42]:
pred_df = pred_df.fillna(pred_df.median())

In [43]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [68]:
from sklearn.model_selection import train_test_split
x = pred_df.drop(columns=["target", "sernr"])
labels = x.columns
y = pred_df["target"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [69]:
clf.fit(x_train, y_train)

RandomForestClassifier()

In [70]:
preds_test = clf.predict(x_test)

In [71]:
(preds_test == y_test).mean()

0.9352135160303312

In [72]:
1 - y_test.mean()

0.9338831980843422

In [75]:
df_imps = pd.DataFrame({"imp": clf.feature_importances_, "feature": labels})

In [76]:
df_imps.sort_values("imp")

,imp,feature
111,0.000000,f49
156,0.000000,f78_6
99,0.000000,f44_8
160,0.000000,f79_4
161,0.000000,f79_5
...,...,...
259,0.017204,altq
181,0.017527,nf88
222,0.017955,nt1
223,0.018318,nt2


In [136]:
col_name = "f1a_5"
col = df[col]
print(col.iloc[0])
print(col.head())
print(col.tail())
print()
print(col.describe())
print()
if len(col.unique()) < 10:
    print("unique distrs:")
    for val in col.unique():
        print(val)
        print((col == val).mean())

0.947
0    0.947
1    0.880
2    1.051
3    2.233
4    0.923
Name: weight, dtype: float64
30063    0.853
30064    1.015
30065    0.787
30066    0.870
30067    0.986
Name: weight, dtype: float64

count    30068.000000
mean         0.999990
std          0.318168
min          0.701000
25%          0.864000
50%          0.917000
75%          0.958000
max          2.241000
Name: weight, dtype: float64

